In [1]:
import pandas as pd
import os
import re
import random
from konlpy.tag import Mecab
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
# torch==1.10.0
# torchtext==0.11.2
# legacy 사용해서 코드 진행하기 위해 버전 맞춰줬음!

import torch
import torchtext
from torchtext.legacy import data, datasets
from torchtext.legacy.data import TabularDataset

import torch.nn as nn
import torch.nn.functional as F


In [3]:
# LSTM 모델 구현
class Net(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(Net, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.lstm = nn.LSTM(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        x, (ho,co) = self.lstm(x)
        h_t = x[:,-1,:] # 모든 문장을 거쳐서 나온 가장 마지막 단어의 출력 값
        self.dropout(h_t)
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [67]:
BATCH_SIZE = 64
vocab_size = 26696
n_classes = 7
DEVICE = torch.device('cpu')
model = Net(4, 128, vocab_size , 128, n_classes, 0.2).to(DEVICE)

In [68]:
model = Net(4, 128, vocab_size , 128, n_classes, 0.2).to(DEVICE)
model.load_state_dict(torch.load("./txtclassification.pt"))
model.eval()

Net(
  (embed): Embedding(26696, 128)
  (dropout): Dropout(p=0.2, inplace=False)
  (lstm): LSTM(128, 128, num_layers=4, batch_first=True)
  (out): Linear(in_features=128, out_features=7, bias=True)
)

In [69]:
tokenizer = Okt()
# 3개의 필드 정의
ID = data.Field(sequential = False,
                use_vocab = False) # 실제 사용은 하지 않을 예정

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=tokenizer.morphs, # 토크나이저로는 Mecab 사용.
                  lower=True,
                  batch_first=True,
                  fix_length=30)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   is_target=True)
# sequential : 시퀀스 데이터 여부. (True가 기본값)
# use_vocab : 단어 집합을 만들 것인지 여부. (True가 기본값)
# tokenize : 어떤 토큰화 함수를 사용할 것인지 지정. (string.split이 기본값)
# lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
# batch_first : 미니 배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지 여부. (False가 기본값)
# is_target : 레이블 데이터 여부. (False가 기본값)
# fix_length : 최대 허용 길이. 이 길이에 맞춰서 패딩 작업(Padding)이 진행된다.

In [70]:
train_data, valid_data, test_data = TabularDataset.splits(
        path='.', train='train_new.csv', validation='valid_new.csv', test='test_new.csv', format='csv',
        fields=[('ID',ID),('text', TEXT), ('label', LABEL)], skip_header=True)

In [71]:
#각각의 단어집합 생성, 2번 이상 등장한 단어들 사용
TEXT.build_vocab(train_data, min_freq=2)
LABEL.build_vocab(train_data)
vocab_size = len(TEXT.vocab)
n_classes = 7
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

단어 집합의 크기 : 26695
클래스의 개수 : 7


In [90]:
# 토치텍스트는 모든 텍스트를 배치 처리하는 것을 지원하고, 단어를 인덱스 번호로 대체하는 BucketIterator를 제공한다.
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (train_data, valid_data, test_data), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False, sort=False) #sorted False 추가!!

In [73]:
def predict(model, test_iter):
    """학습된 모델로 정답 파일 예측"""
    model.eval()
    preds = []
    pred = []
    for batch in test_iter:
        x = batch.text.to(DEVICE)
        logit = model(x)
        pred=torch.argmax(logit,axis= 1).tolist()
        preds.extend(pred)
    return preds 

In [74]:
pred = predict(model,test_iter)
len(pred)

13491

In [116]:
test_value_counts = {
    0:0, 1:0, 2:0,3:0,4:0,5:0,6:0
}

for tmp in pred:
    test_value_counts[tmp] +=1
    
print(test_value_counts)

{0: 1598, 1: 1604, 2: 1728, 3: 1871, 4: 792, 5: 891, 6: 5007}


In [76]:
submission_df = pd.read_csv('./output/all_zero_submission.csv')
submission_df.head()

,ID,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [77]:
submission_df['label']= pred

In [78]:
submission_df.head()

,ID,label
0,0,0
1,1,2
2,2,6
3,3,2
4,4,0


In [80]:
submission_df.to_csv('./lora.csv',index=False)

In [100]:
def evaluate(model, val_iter):
    real_answer_list = []
    my_predict_list = []
    """검증 데이터셋 평가"""
    model.eval()
    batch_cor, total_loss = 0, 0

    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y= torch.tensor(y,dtype = torch.long,device =DEVICE)
        logit = model(x)
        real_answer_list.extend(y.data)
        loss = F.cross_entropy(logit, y)
        my_predict_list.extend(logit.max(1)[1])
        total_loss += loss.item()
        batch_cor += (logit.max(1)[1] == y.data).sum() 

    size = len(val_iter.dataset)
    avg_loss = total_loss / size 
    avg_accuracy = 100.0 * batch_cor / size


    return avg_loss, avg_accuracy,real_answer_list,my_predict_list

In [101]:
val_loss, val_accuracy, valid_answer_list, my_predict_list= evaluate(model, val_iter)

C:\Users\user\AppData\Local\Temp\ipykernel_27396\1864612902.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y= torch.tensor(y,dtype = torch.long,device =DEVICE)


In [102]:
print(len(valid_answer_list))
print(len(my_predict_list))


13180
13180


In [108]:
valid_answer_list = [tmp.item() for tmp in valid_answer_list]
my_predict_list = [tmp.item() for tmp in my_predict_list]

In [83]:
print(val_loss, val_accuracy)

0.014964916187371035 tensor(69.5068)


In [114]:
import numpy as np
from sklearn.metrics import f1_score

def calculate_f1_score(y_true, y_pred, average=True):
    """각 라벨별 F1 score를 계산하고, 평균을 구해 반환"""
    f1_scores = []
    
    y_true = np.array(y_true) # list를 numpy array로 변환
    y_pred = np.array(y_pred) # list를 numpy array로 변환
    
    for label in range(7):
        y_true_label = (y_true == label).astype(int)
        y_pred_label = (y_pred == label).astype(int)
        f1 = f1_score(y_true_label, y_pred_label)
        f1_scores.append(f1)
    if average:
        return sum(f1_scores) / len(f1_scores)
    else:
        return f1_scores

In [115]:
calculate_f1_score(valid_answer_list,my_predict_list)

0.663194577030277